### Importing Libraries

In [11]:
# Importing Libraries

import numpy as np
import pandas as pd


import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#------ Additional Lib
import neattext as nt
import neattext.functions as nfx

from sklearn.feature_extraction.text import TfidfVectorizer   # Turning textual data into numeric for computation
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder               # For encoding categorical target attr
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn import svm   # Baseline

# ------- Validation metrics
from sklearn.metrics import accuracy_score    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss                  
from sklearn.metrics import classification_report

### Data Preprocessing

In [ ]:
nltk.download('stopwords')

def get_train_dataset():
    return pd.read_csv('data_train.csv', header=None, names=["country", "sku_id", "title", "category_lvl1","category_lvl2",
                                                       "category_lvl3", "description", "price", "type"])  

def preprocess(content):
    ps = PorterStemmer()
    CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    # Using str(content) because there are some float values in combined
    stemmed_content = re.sub('[^a-zA-Z]',' ',str(content))   # Dropping all encodings, numbers etc
    stemmed_content = re.sub(CLEANR, '',stemmed_content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


def feature_select(df):
    df['combined'] = df['title']+" "+df['description']
    df['combined']
    return df

def print_missing_stats(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

    
    

train_df = get_train_dataset()
train_df = feature_select(train_df)
train_df['combined'] = train_df['combined'].apply(preprocess)



[nltk_data] Downloading package stopwords to C:\Users\Computer
[nltk_data]     Point\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def extract_features(df):
    # Extract features prior to encoding to retain categorical data
    X = df['combined']
    Y1 = df['category_lvl1']
    Y2 = df['category_lvl2']
    Y3 = dF['category_lvl3']
    return X,Y1,Y2,Y3


def print_missing_stats(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

    

def encode_utility(data):
    encoder = LabelEncoder()
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
#     impute_ordinal = pd.get_dummies(impute_reshape).values # one hot encoding   #Not working
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

def encode(target):
    for columns in target:
        encode_utility(train_df[columns])


#     Y_c1 = pd.get_dummies(tCol_df['c1']).values 
#     print('Shape of label tensor:', train_Y_c1.shape)
    
def impute(df):
    imputer = KNNImputer(n_neighbors = 5)
    df_imputed = imputer.fit_transform(train_df)
    df_imputed
    
    
    
    
    
# X, Y1, Y2, Y3 = extract_features(train_df)  After 
print_missing_stats(train_df)
encode(['category_lvl1', 'category_lvl2', 'category_lvl3'])
train_df = impute(train_df)
print_missing_stats(train_df)




In [20]:
train_df

,country,sku_id,title,category_lvl1,category_lvl2,category_lvl3,description,price,type,combined
0,my,AD674FAASTLXANMY,Adana Gallery Suri Square Hijab – Light Pink,2,56,113,<ul><li>Material : Non sheer shimmer chiffon</...,49.00,local,adana galleri suri squar hijab light pink ul l...
1,my,AE068HBAA3RPRDANMY,Cuba Heartbreaker Eau De Parfum Spray 100ml/3.3oz,3,3,67,Formulated with oil-free hydrating botanicals/...,128.00,international,cuba heartbreak eau de parfum spray ml oz form...
2,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,7,1,95,<ul> <li>150cm mini microphone compatible for ...,25.07,international,andoer cm cellphon smartphon mini dual head om...
3,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,3,23,147,<ul> <li>ANMYNA Complaint Silky Set (Shampoo 5...,118.00,local,anmyna complaint silki set shampoo ml conditio...
4,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,3,36,23,<ul> <li>100% Authentic</li> <li>Rrefresh and ...,114.80,international,argit argiltubo green clay face bodi ml ul li ...
...,...,...,...,...,...,...,...,...,...,...
36278,sg,SA584ELAA4G4W0SGAMZ,SADES K10 LED Backlit Wired USB Mechanical Gam...,1,8,82,<ul> <li>No driver needed.Blue Switches is the...,67.09,international,sade k led backlit wire usb mechan game keyboa...
36279,sg,SO499HAAA4CCLASGAMZ,SONA 20L Electric Oven SEO 2220,5,30,109,<ul> <li>&nbsp;2 Years Warranty<br></li> <li>T...,69.00,local,sona l electr oven seo ul li nbsp year warrant...
36280,sg,TI990ELAA5ZV1JSGAMZ,OP1001 Portable Wireless Bluetooth 2.1 Speaker...,1,8,155,"<ul> <li>With colorful lights on the button, l...",35.00,international,op portabl wireless bluetooth speaker hand fre...
36281,sg,WO203HLAA3KFPCSGAMZ,Woot-Woot TicTacToe Pillow Case (White),4,5,126,<ul> <li>100% Cotton</li> <li>Safe for Sensiti...,23.00,local,woot woot tictacto pillow case white ul li cot...
